### try
https://www.benfrederickson.com/matrix-factorization/

In [10]:
import pandas as pd
from scipy.sparse import coo_matrix, linalg
import numpy
from implicit.nearest_neighbours import (
    bm25_weight,
)
from implicit.datasets.lastfm import get_lastfm

In [11]:
artists, users, plays = get_lastfm()
plays = plays.tocsr()

184MB [01:20, 2.29MB/s]                           


In [12]:
artist_factors, _, user_factors = linalg.svds(bm25_weight(plays), 50)

In [13]:
class TopRelated(object):
    def __init__(self, artist_factors):
        # fully normalize artist_factors, so can compare with only the dot product
        norms = numpy.linalg.norm(artist_factors, axis=-1)
        self.factors = artist_factors / norms[:, numpy.newaxis]

    def get_related(self, artistid, N=10):
        scores = self.factors.dot(self.factors[artistid])
        best = numpy.argpartition(scores, -N)[-N:]
        return sorted(zip(best, scores[best]), key=lambda x: -x[1])

In [14]:
tr = TopRelated(artist_factors)
tr.get_related(12, N=5)

[(12, 1.0000000000000002),
 (79547, 0.9991338615841725),
 (286724, 0.999131341780894),
 (287417, 0.9988966941116743),
 (27373, 0.9988018733856024)]